In [ ]:
# Generic
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings, gc
warnings.filterwarnings("ignore")

# Tensorflow
import tensorflow as tf

# ktrain
import ktrain
from ktrain import text

# sklearn
from sklearn.model_selection import train_test_split


In [2]:
from tensorflow import keras
tf.__version__

'2.5.0'

In [2]:
df = pd.read_csv("TrainingDataForBERTMODEL-Structure-Binary.csv", header='infer')
# Dropping Null Values
df.dropna(inplace=True)

# Total Records
print("Total Records: ", df.shape[0])

# Inspect
df.head()


Total Records:  2038


,Topic,Sentence
0,Structure,Begins with a story to engage audience
1,No,He clasps his hands a lot while he speaks
2,No,Use of humour
3,No,Use of analogy
4,No,He makes good eye contact


In [3]:
# Data Split
target = ['Topic']
data = ['Sentence']

X = df[data]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.1, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1834, 1) (204, 1) (1834, 1) (204, 1)


In [4]:
# Common Parameters
max_len = 500
batch_size = 16
learning_rate = 5e-5
epochs = 5

In [5]:
#########
## With Transformer = bert-base-uncased
######
# Transformer Model
model_ = 'bert-base-uncased'
t_mod = text.Transformer(model_, maxlen=500, classes = [0,1])


'''Converting split data to list [so it can processed]'''
#train
X_tr = X_train['Sentence'].tolist()
y_tr = y_train['Topic'].tolist()

#test
X_ts = X_test['Sentence'].tolist()
y_ts = y_test['Topic'].tolist()


# Pre-processing training & test data
train = t_mod.preprocess_train(X_tr,y_tr)
test = t_mod.preprocess_train(X_ts,y_ts)

# Model Classifier
model = t_mod.get_classifier()

learner = ktrain.get_learner(model, train_data=train, val_data=test, batch_size=batch_size)

preprocessing train...
language: en
train sequence lengths:
	mean : 11
	95percentile : 28
	99percentile : 52


Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 12
	95percentile : 25
	99percentile : 54


Is Multi-Label? False


In [6]:
# Train Model
learner.fit_onecycle(learning_rate, epochs)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/5
115/115 [==============================] - 13838s 120s/step - loss: 0.5240 - accuracy: 0.7743 - val_loss: 0.3728 - val_accuracy: 0.8284
Epoch 2/5
115/115 [==============================] - 13421s 117s/step - loss: 0.3496 - accuracy: 0.8441 - val_loss: 0.2647 - val_accuracy: 0.8922
Epoch 3/5
115/115 [==============================] - 12800s 111s/step - loss: 0.2208 - accuracy: 0.9149 - val_loss: 0.2607 - val_accuracy: 0.9020
Epoch 4/5
115/115 [==============================] - 12372s 108s/step - loss: 0.1016 - accuracy: 0.9607 - val_loss: 0.2445 - val_accuracy: 0.8873
Epoch 5/5
115/115 [==============================] - 12180s 106s/step - loss: 0.0346 - accuracy: 0.9875 - val_loss: 0.3236 - val_accuracy: 0.8873


In [8]:
# Evaluate
x = learner.validate(class_names=t_mod.get_classes())

              precision    recall  f1-score   support

          No       0.94      0.95      0.94       169
   Structure       0.74      0.71      0.72        35

    accuracy                           0.91       204
   macro avg       0.84      0.83      0.83       204
weighted avg       0.91      0.91      0.91       204



In [12]:

ktrain.get_predictor(learner.model,preproc=t_mod).save('StructureBinaryClassifier')

In [7]:
#old videos
df1=pd.read_csv("PresentationSkillDataSetForBert.csv")
#classes = ['Delivery','Structure','Visual Aids']
classes = ['Structure','']
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)

segmentId=df1['SegmentId']
segList=list(segmentId)

startTime=df1['StartTime']
strList=list(startTime)

endTime=df1['EndTime']
endList=list(endTime)

transcript=df1['Transcript']
transList=list(transcript)
transList1=[]
for i in transList:
    if i != 'nan':
        transList1.append(i)

In [ ]:
i=0
segTranscript=[]
allSegmentsTrans=[]
startime=strList[0]
endTime=endList[0]
segTrigger=i
while i < len(segList)-2:
    #print(segList[i])
    #print()
    if segList[i]==segList[i+1]:
        print(segList[i],'  ==  ',segList[i+1])
        print()
        segTranscript.append(transList1[i])
        #segTranscript.append(transList1[i+1])
        print( transList1[i])
        #print( transList1[i+1])
        print()
        i+=1
    else:
        print(segList[i],'  !=  ',segList[i+1])
        #print()
        #print('start time= ',strList[i+1],'    ','end time= ',endList[i+1])

        segTranscript.append(transList1[i])
        listToString=' '.join([str(item) for item in segTranscript ])
        allSegmentsTrans.append([segList[i],startime,endTime,segTranscript,listToString])
        print(listToString)
        print()
        segTranscript=[]
        
        startime=strList[i+1]
        endTime=endList[i+1]
        
        i+=1

In [1]:
#### 
# Write the annotation result to Excel File- old videos

import xlrd
import xlsxwriter
workbook = xlsxwriter.Workbook('PresentationCorpusAnnotatedUsingBertBaseUncase-BinaryStructure.xlsx')
worksheet = workbook.add_worksheet('Sheet1')
worksheet.write('A1', 'SegmentId')
worksheet.write('B1', 'StartTime')
worksheet.write('C1', 'EndTime')
worksheet.write('D1', 'Transcript')
worksheet.write('E1', 'Topic')
row=1
col=0
transRow=0

for i in allSegmentsTrans:
    print(i[0])
    print(i[4])
    print()
    print('row= ',row)
    print()
    mainRow=row
    worksheet.write(row,col,i[0])
    worksheet.write(row,col+1,i[1])
    worksheet.write(row,col+2,i[2])
    transRow=row
    for j in i[3]:
        worksheet.write(row,col+3,j)
        row+=1
    print('prediction= ',predictor.predict(i[4]))
    print()
    worksheet.write(mainRow,col+4,predictor.predict(i[4]))
        
        

    
workbook.close()   


NameError: name 'transList1' is not defined

In [11]:
# characterise segments with full stop
df1=pd.read_csv("Pres Videos To 6 sentences using Full Stop as Delimiter.csv")
#classes = ['Delivery','Structure','Visual Aids']
classes = ['Structure','']
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)

segmentId=df1['SegmentId']
segList=list(segmentId)

startTime=df1['StartTime']
strList=list(startTime)

endTime=df1['EndTime']
endList=list(endTime)

transcript=df1['Transcript']
transList=list(transcript)
transList1=[]
for i in transList:
    print(i)
    print('*********')
    if i != 'nan':
        transList1.append(i)

Pay practitioner today, I'm going to be
*********
teaching you with the help of my friend,
*********
Dana Blouin, the top seven tip for a
*********
great presentation,
*********
Dana blue and a an entrepreneur
*********
currently living in Bangkok, Thailand,
*********
where he is a co and CEO of a
*********
company called mango lab. Anyways here's
*********
some of his tip. We all have to give
*********
presentation at different point in our
*********
life that might be for school or work or
*********
an organization that you weren't
*********
involved with no matter where We give
*********
the presentation, or to who we want to
*********
make sure that it ha the impact on the
*********
Audience and that it memorable, these
*********
are seven tip that are going to ensure
*********
that you nail the presentation, and leave
*********
a lasting impression on your audience.
*********
Number one know your audience. The first
*********
step in preparing for any presentation
*********
should

*********
after hearing to this particular module
*********
you all go out there and make some
*********
effective and beautiful presentation
*********
and really bow your audience Thank you
*********
So much for listening to u and we Do
*********
hope, you like this, video please do nt
*********
  forget to share your comment. thank you
*********
  forget to share your comment. thank you
*********
  once again,
*********
Hello everyone ! Welcome to public speaking .
*********
Today ’ s first presenter is Ranjit .
*********
And I would ask everyone to give him their full attention .
*********
Chantal could you dim the light please ?
*********
The British imposed a lot of unpopular tax on the American colonist
*********
and didn ’ t let them elect representative to the British government .
*********
In retaliation the Americans started to boycott British good .
*********
In 1773 Americans in Massachusetts dumped a shipload of
*********
British tea into Boston Harbor .
*********
This bec

Imagine you have to give an important presentation in English tomorrow .
*********
How would you feel about it ?
*********
This lesson will help you learn useful phrase and technique to introduce yourself and your
*********
topic keep your idea organised deal with problem and respond to question from audience
*********
member .
*********
Imagine you ’ re standing in front of your colleague .
*********
You need to introduce yourself and what your presentation is about .
*********
What are some word and phrase you could use ?
*********
If some people in the audience don ’ t know who you are you should introduce yourself
*********
and your position .
*********
In a more formal setting you could say something like this :
*********
Good morning everyone .
*********
For those who don ’ t know me my name ’ s Simon and I work in the marketing department .
*********
Or : Hello everybody .
*********
Before we begin let me introduce myself briefly : I ’ m Reese and I ’ m the head of HR .
********

else. S work, so a I just mentioned, do
*********
not have them too long that will eat
*********
into your word count and ultimately it
*********
will affect the mark that you ll get. So,
*********
paraphrasing is what we re looking to
*********
see that you re doing your assignment.
*********
However, anything is when you look at
*********
other people, s work. and you put it into
*********
your own word. It wa more to
*********
paraphrasing than just changing one or
*********
two word here and there or cutting out
*********
word from a sentence. It will show your
*********
understanding to the reader, so, instead
*********
of just identifying that something s
*********
important and copying it word
*********
You ve been able to read it understand
*********
it and apply it to your work. So I m
*********
going to show you an example, of how to
*********
go from using a quote to paraphrasing it
*********
So if We look at this quote here in
*********
academic writing, it s essential to s

*********
which is short for Scalable Vector Graphics
*********
you can scale up without pixelation .
*********
SVGs can be resized from small to large
*********
without losing quality because they consist of
*********
mathematical line and fill instead of pixel .
*********
They are also much smaller in size .
*********
For example the icon option in the insert tab are SVGs
*********
and recent version of PowerPoint
*********
allows you to edit these graphic .
*********
You can use the eyedropper tool to change the fill
*********
or you can even disassemble the SVG image by converting it
*********
to an Office shape .
*********
This way you can change the fill of individual part
*********
of the image .
*********
So whenever you can choose vector graphic
*********
over raster image .
*********
You can scale them up without loss of resolution
*********
and they are smaller in size a well .
*********
Number five apply the rule of third .
*********
In photography and cinematography
******

*********
with all the data from the new time period
*********
from the new month .
*********
In these situation you 're very frequently faced with
*********
the decision to whether you want to re now
*********
all the analysis
*********
to incorporate this new information or whether
*********
will you leave all the analysis as ?
*********
And then just include a bubble on the page
*********
stating the limitation or stating what ha not
*********
yet been included in the analysis .
*********
Bubbles are so important because very frequently
*********
instead of re the whole thing re analysis
*********
re all kind of page .
*********
Problems like that you can frequently use with the bubble .
*********
And this is why there is a magic saying in consulting
*********
and this go like  If in trouble use a bubble ! ''
*********
So if you are struggling with something
*********
something just is n't right .
*********
You just need to use this statement
*********
just need to make some correct

first thing I wanted to share with you
*********
this morning.
*********
the point is: make it easy for your
*********
listener to follow your story, your
*********
outline will serve a guide post along
*********
the way
*********
you also notice that, during his
*********
presentation. Jobs us word like
*********
extraordinary amazing and cool. He is
*********
passionate, enthusiastic and it show
*********
incredible unbelievable amazing awesome
*********
extraordinary year forever.
*********
You know your audience want to be wowed
*********
not put to sleep too many people fall
*********
into this presentation mode. If stiff is
*********
formal, it lack pizzazz, we your
*********
listener are giving you permission to
*********
have fun and to be excited about your
*********
company, your product or your service. If
*********
you not passionate about it we not
*********
going to be remember. Job selling
*********
hardware selling an experience. If
*********
you offer number and statis

In [16]:
i=0
segTranscript=[]
allSegmentsTrans=[]
startime=strList[0]
endTime=endList[0]
segTrigger=segList[i]
while i < len(segList)-2:
    #print(segList[i])
    #print()
    if segList[i+1]==str('nan'):
        #print(segList[i],'  ==  ',segList[i+1])
        #print()
        segTranscript.append(transList1[i])
        #segTranscript.append(transList1[i+1])
        print( transList1[i])
        #print( transList1[i+1])
        print()
        i+=1
    else:
        print(segList[i],'  !=  ',segList[i+1])
        #print()
        #print('start time= ',strList[i+1],'    ','end time= ',endList[i+1])

        segTranscript.append(transList1[i])
        listToString=' '.join([str(item) for item in segTranscript ])
        allSegmentsTrans.append([segTrigger,startime,endTime,segTranscript,listToString])
        print(listToString)
        print()
        segTranscript=[]
        
        startime=strList[i+1]
        endTime=endList[i+1]
        segTrigger=segList[i+1]
        i+=1
        

MnIPpUiTcRc1   !=   nan
Pay practitioner today, I'm going to be

nan   !=   nan
teaching you with the help of my friend,

nan   !=   nan
Dana Blouin, the top seven tip for a

nan   !=   nan
great presentation,

nan   !=   nan
Dana blue and a an entrepreneur

nan   !=   nan
currently living in Bangkok, Thailand,

nan   !=   nan
where he is a co and CEO of a

nan   !=   MnIPpUiTcRc2
company called mango lab. Anyways here's

MnIPpUiTcRc2   !=   MnIPpUiTcRc3
some of his tip. We all have to give

MnIPpUiTcRc3   !=   nan
presentation at different point in our

nan   !=   nan
life that might be for school or work or

nan   !=   nan
an organization that you weren't

nan   !=   nan
involved with no matter where We give

nan   !=   nan
the presentation, or to who we want to

nan   !=   nan
make sure that it ha the impact on the

nan   !=   nan
Audience and that it memorable, these

nan   !=   nan
are seven tip that are going to ensure

nan   !=   nan
that you nail the presentation, and leave

na

to prepare and deliver fantastic presentation of all kind

nan   !=   yoD8RMq2OkU103
from TED talk to the boardroom .

yoD8RMq2OkU103   !=   nan
But more than presentation they ’ ve used it also for email

nan   !=   nan
for proposal for preparing interview for preparing conference …

nan   !=   nan
Basically it work in any situation

nan   !=   yoD8RMq2OkU104
where you need to change what people believe feel and do .

yoD8RMq2OkU104   !=   yoD8RMq2OkU105
You really don ’ t need a magic wand .

yoD8RMq2OkU105   !=   yoD8RMq2OkU106
This is all the magic you need .

yoD8RMq2OkU106   !=   yoD8RMq2OkU107
Thank you .

yoD8RMq2OkU107   !=   yoD8RMq2OkU108
Thank you . So now I need to give you a call to action and here it is .

yoD8RMq2OkU108   !=   yoD8RMq2OkU109
nan

yoD8RMq2OkU109   !=   nan
Remember those three magic ingredient of every great presentation :

nan   !=   nan
the audience the speaker

nan   !=   yoD8RMq2OkU110
and the transformation you want to create in your audience .

yoD


nan   !=   nan
when it come to giving a presentation

nan   !=   nan
in a room full of people? it is not an

nan   !=   ADJAcyTq1us9
easy task, I must say when I started.

ADJAcyTq1us9   !=   ADJAcyTq1us10
training a couple of year back. I

ADJAcyTq1us10   !=   nan
actually felt another breakdown because

nan   !=   nan
of the fact that I had to give a

nan   !=   nan
presentation in front of a room of

nan   !=   nan
people? I felt dizzy a Well, a nervous

nan   !=   ADJAcyTq1us11
and my hand started to sweat. Well, this,

ADJAcyTq1us11   !=   nan
is a phenomenon which happens with most

nan   !=   nan
of the people They get a nightmare about

nan   !=   nan
giving a presentation you Have a fear of

nan   !=   nan
phobia, and that is something which we

nan   !=   nan
will challenge today in our

nan   !=   ADJAcyTq1us12
datian. Well, we Have to define what

ADJAcyTq1us12   !=   nan
exactly is a presentation let s look at

nan   !=   nan
that it is a process of presenting the

nan   

He wa reading from his paper so it didn ’ t feel like he wa really talking to u you know ?

nan   !=   V8eLdbKXGzk17
Exactly . It ’ s important when you ’ re speaking

V8eLdbKXGzk17   !=   V8eLdbKXGzk18
to look up every sentence or two and make eye contact with someone in the audience .

V8eLdbKXGzk18   !=   nan
Anything else that Ranjit could do to appear more relaxed and feel more confident ?

nan   !=   V8eLdbKXGzk19
He could smile more .

V8eLdbKXGzk19   !=   V8eLdbKXGzk20
Yes ! Smiling make u feel relaxed .

V8eLdbKXGzk20   !=   nan
Oh and make sure your phone is off before you speak next time !

nan   !=   V8eLdbKXGzk21
I do have one more suggestion .

V8eLdbKXGzk21   !=   nan
To connect with listener it ’ s a great idea to use a

nan   !=   V8eLdbKXGzk22
personal story or experience a an example .

V8eLdbKXGzk22   !=   nan
Can you try that next time ?

nan   !=   V8eLdbKXGzk23
I think so yes . Next week my presentation will be about the cause and effect of stress .

V8eLdbKXGzk2

hMk5s1y486I12   !=   hMk5s1y486I13
It ll. Keep your audience locked in from

hMk5s1y486I13   !=   nan
start to finish

nan   !=   hMk5s1y486I14
hello world. My name is Mike Lauder here

hMk5s1y486I14   !=   nan
with viz me and if you stick around

nan   !=   nan
not only will I tell you these seven

nan   !=   nan
different way to structure your

nan   !=   hMk5s1y486I15
presentation. I ll. also finish my story.

hMk5s1y486I15   !=   hMk5s1y486I16
nan

hMk5s1y486I16   !=   hMk5s1y486I17
nan

hMk5s1y486I17   !=   nan
and give you something to laugh about

nan   !=   nan
before we get to the seven different

nan   !=   hMk5s1y486I18
way to structure your presentation. it s

hMk5s1y486I18   !=   nan
important to thoroughly plan your

nan   !=   hMk5s1y486I19
message in a rough draft. First, you got

hMk5s1y486I19   !=   nan
to decide if you want your message to be

nan   !=   nan
informative, inspiring, persuasive or

nan   !=   nan
entertaining it can be one or two of

nan   !=   nan
the

...

Iwpi1Lm6dFo70   !=   Iwpi1Lm6dFo71
We are extremely limited to understanding more .

Iwpi1Lm6dFo71   !=   Iwpi1Lm6dFo72
Let 's move on . Go to working memory .

Iwpi1Lm6dFo72   !=   Iwpi1Lm6dFo73
nan

Iwpi1Lm6dFo73   !=   nan
I 've already given you this bad vibe that your working memory is bad

nan   !=   Iwpi1Lm6dFo74
and I 'm afraid I 'm not coming with better news .

Iwpi1Lm6dFo74   !=   Iwpi1Lm6dFo75
I 'm coming with worse news and it go like this .

Iwpi1Lm6dFo75   !=   nan
This equation ha the basis of John Sweller and Mayer

nan   !=   nan
and they come to the conclusion

nan   !=   nan
that there is something in our brain called 'the redundancy effect '

nan   !=   Iwpi1Lm6dFo76
and it work like this .

Iwpi1Lm6dFo76   !=   nan
If you have text sentence on your PowerPoint

nan   !=   nan
and you persist with the annoying idea of speaking at the same time

nan   !=   nan
what will be remembered by the audience

nan   !=   Iwpi1Lm6dFo77
is zero .

Iwpi1Lm6dFo77   !=   Iwpi1Lm6dF

PgOD1j2DhNg108   !=   nan
Let 's now look into another

nan   !=   PgOD1j2DhNg109
very ambiguous chart element .

PgOD1j2DhNg109   !=   PgOD1j2DhNg110
This is a so callout .

PgOD1j2DhNg110   !=   PgOD1j2DhNg111
This is this element right here .

PgOD1j2DhNg111   !=   nan
So this is an element that you can just

nan   !=   PgOD1j2DhNg112
include in a chart .

PgOD1j2DhNg112   !=   PgOD1j2DhNg113
Gives detail about this specific element .

PgOD1j2DhNg113   !=   nan
So a I wrote here it can include remark

nan   !=   nan
on specific element of a chart or other relevant

nan   !=   PgOD1j2DhNg114
contribution that you wan na make .

PgOD1j2DhNg114   !=   nan
A callout is something that you can easily use

nan   !=   nan
just add detail

nan   !=   nan
add information

nan   !=   PgOD1j2DhNg115
add context to one specific element .

PgOD1j2DhNg115   !=   PgOD1j2DhNg116
This is a frequent way how you can do it .

PgOD1j2DhNg116   !=   nan
So this element just consists of one line

nan   != 

nan   !=   nan
what are your little tip ?

nan   !=   nan
What are the thing that you 've frequently use

nan   !=   nan
to improve your presentation ?

nan   !=   nan
Let 's have a conversation here from which

nan   !=   PgOD1j2DhNg246
everybody can benefit .

PgOD1j2DhNg246   !=   nan
Again if you took any value out of this video at all

nan   !=   PgOD1j2DhNg247
please hit the like button for the YouTube algorithm .

PgOD1j2DhNg247   !=   nan
This would really help me and my channel to improve

nan   !=   nan
to grow and to tell the YouTube algorithm that this

nan   !=   PgOD1j2DhNg248
is a video worthwhile for other distributing .

PgOD1j2DhNg248   !=   nan
If you are a first time viewer to this channel

nan   !=   PgOD1j2DhNg249
thank you for your time and thank you for watching .

PgOD1j2DhNg249   !=   PgOD1j2DhNg250
Please hit the subscribe button you will stay up .

PgOD1j2DhNg250   !=   PgOD1j2DhNg251
If you want to see more content like this .

PgOD1j2DhNg251   !=   nan
And

nan   !=   Hp7Id3Yb9XQ289
I can actually use the visuals to explain my answer . And that turn out to be very helpful . So I really recommend that at the end of your talk

Hp7Id3Yb9XQ289   !=   Hp7Id3Yb9XQ290
nan

Hp7Id3Yb9XQ290   !=   Hp7Id3Yb9XQ291
during Q and A put your summary slide back up again . It 's helpful for you and for the people attending your talk .

Hp7Id3Yb9XQ291   !=   Hp7Id3Yb9XQ292
nan

Hp7Id3Yb9XQ292   !=   Hp7Id3Yb9XQ293
Ok so what have I just told you about organizing a great talk ? First of all use PowerPoint wisely .

Hp7Id3Yb9XQ293   !=   Hp7Id3Yb9XQ294
Have very clean minimal slide . Secondly start out with a broad introduction so that everyone in the audience

Hp7Id3Yb9XQ294   !=   nan
understands the big question that are compelling in your field and then introduce the specific gradually

nan   !=   Hp7Id3Yb9XQ295
giving them just enough background information so that they can follow the data but not so much that they 're overwhelmed .

Hp7Id3Yb9XQ295   != 

Yl_FJAOcFgQ28   !=   Yl_FJAOcFgQ29
will be left with after you stop. If

Yl_FJAOcFgQ29   !=   nan
you ever been to a firework display,

nan   !=   Yl_FJAOcFgQ30
you know. that the biggest brightest

Yl_FJAOcFgQ30   !=   Yl_FJAOcFgQ31
firework are usually saved. For the end

Yl_FJAOcFgQ31   !=   nan
this doe mean you have to finish

nan   !=   nan
with a bang, but you do want to leave a

nan   !=   nan
lasting impression

nan   !=   Yl_FJAOcFgQ32
watch. These presenter clinch the

Yl_FJAOcFgQ32   !=   Yl_FJAOcFgQ33
clothes, to summarise. Whenever we have

Yl_FJAOcFgQ33   !=   nan
offered bonus to incentivize our staff

nan   !=   nan
in sale, HR and manufacturing division,

nan   !=   nan
productivity ha increased in some case

nan   !=   Yl_FJAOcFgQ34
quite dramatically, but a we saw In R & D.

Yl_FJAOcFgQ34   !=   nan
Introducing pay by performance ha had

nan   !=   Yl_FJAOcFgQ35
precisely the opposite effect.

Yl_FJAOcFgQ35   !=   nan
Incentivized research unit were, on

nan   !=   

In [17]:
for i in allSegmentsTrans:
    print(i)
    print()

['MnIPpUiTcRc1', 0.01, 0.14, ["Pay practitioner today, I'm going to be"], "Pay practitioner today, I'm going to be"]

[nan, nan, nan, ['teaching you with the help of my friend,'], 'teaching you with the help of my friend,']

[nan, nan, nan, ['Dana Blouin, the top seven tip for a'], 'Dana Blouin, the top seven tip for a']

[nan, nan, nan, ['great presentation,'], 'great presentation,']

[nan, nan, nan, ['Dana blue and a an entrepreneur'], 'Dana blue and a an entrepreneur']

[nan, nan, nan, ['currently living in Bangkok, Thailand,'], 'currently living in Bangkok, Thailand,']

[nan, nan, nan, ['where he is a co and CEO of a'], 'where he is a co and CEO of a']

[nan, nan, nan, ["company called mango lab. Anyways here's"], "company called mango lab. Anyways here's"]

['MnIPpUiTcRc2', 0.14, 0.17, ['some of his tip. We all have to give'], 'some of his tip. We all have to give']

['MnIPpUiTcRc3', 0.17, 0.34, ['presentation at different point in our'], 'presentation at different point in our']



[nan, nan, nan, ['you Do not know the answer to that'], 'you Do not know the answer to that']

[nan, nan, nan, ['question It s absolutely alright, we are'], 'question It s absolutely alright, we are']

[nan, nan, nan, ['human being and we are not robot. We'], 'human being and we are not robot. We']

['ADJAcyTq1us122', 14.53, 15.05, ['are not supposed to know all the answer'], 'are not supposed to know all the answer']

[nan, nan, nan, ['hence, if you Do nt know a particular'], 'hence, if you Do nt know a particular']

[nan, nan, nan, ['answer to a question apologize and say'], 'answer to a question apologize and say']

[nan, nan, nan, ['you know What I Do nt know the answer to'], 'you know What I Do nt know the answer to']

[nan, nan, nan, ['that question but I will find that out'], 'that question but I will find that out']

[nan, nan, nan, ['for you last but not the least control'], 'for you last but not the least control']

[nan, nan, nan, ['hijacker. You might have somebody in'], '


[nan, nan, nan, ['Here are three technique you can use .'], 'Here are three technique you can use .']

['fzIxD1jXn4441', 5.18, 5.18, ['One : establish a problem which many people in your audience have .'], 'One : establish a problem which many people in your audience have .']

['fzIxD1jXn4442', 5.23, 5.23, ['Then establish that you have a solution to their problem .'], 'Then establish that you have a solution to their problem .']

['fzIxD1jXn4443', 5.28, 5.41, ['For example :'], 'For example :']

[nan, nan, nan, ['Have you ever felt unfairly treated at work or felt that the work you do isn ’ t appreciated ?'], 'Have you ever felt unfairly treated at work or felt that the work you do isn ’ t appreciated ?']

[nan, nan, nan, ['We ’ ve been working to design new HR policy that will make sure all staff get fair recognition'], 'We ’ ve been working to design new HR policy that will make sure all staff get fair recognition']

[nan, nan, nan, ['for their contribution to the company .'], 'for


[nan, nan, nan, ['conference room it s going to be an'], 'conference room it s going to be an']

[nan, nan, nan, ['office? Where Is What is the venue? for'], 'office? Where Is What is the venue? for']

[nan, nan, nan, ['the presentation, and time? How long will'], 'the presentation, and time? How long will']

[nan, nan, nan, ['you have? Will you have? a 20 minute'], 'you have? Will you have? a 20 minute']

[nan, nan, nan, ['presentation? Is this a one hour keynote?'], 'presentation? Is this a one hour keynote?']

[nan, nan, nan, ['Is it 90 minute you have? a half an hour'], 'Is it 90 minute you have? a half an hour']

[nan, nan, nan, ['just you need to know what time? How much'], 'just you need to know what time? How much']

[nan, nan, nan, ['time you have? and what takeaway are the'], 'time you have? and what takeaway are the']

[nan, nan, nan, ['client, or the people bringing you in'], 'client, or the people bringing you in']

[nan, nan, nan, ['expecting? What are you going, to leav

[nan, nan, nan, ['while working on several client presentation .'], 'while working on several client presentation .']

['PgOD1j2DhNg2', 0.09, 0.19, ['By doing that I learned lot of little tip and trick'], 'By doing that I learned lot of little tip and trick']

[nan, nan, nan, ['specific element'], 'specific element']

[nan, nan, nan, ['that consultant use to give their'], 'that consultant use to give their']

[nan, nan, nan, ['presentation the special professional look that make it'], 'presentation the special professional look that make it']

[nan, nan, nan, ['ready for board room and just distinctive .'], 'ready for board room and just distinctive .']

['PgOD1j2DhNg3', 0.21, 0.27, ['When I flip through a presentation I can directly'], 'When I flip through a presentation I can directly']

[nan, nan, nan, ['tell you if this is a presentation from an MBB'], 'tell you if this is a presentation from an MBB']

[nan, nan, nan, ['consulting firm .'], 'consulting firm .']

['PgOD1j2DhNg4', 0.

[nan, nan, nan, ['from Providence to Aspen to see about a girl'], 'from Providence to Aspen to see about a girl']

[nan, nan, nan, ['and everything that can go wrong doe go wrong'], 'and everything that can go wrong doe go wrong']

[nan, nan, nan, ['on their way .'], 'on their way .']

['h6sm47j-Am443', 4.16, 4.24, ['And so all these mishap become very humorous'], 'And so all these mishap become very humorous']

[nan, nan, nan, ["because you know there 's a goal in mind"], "because you know there 's a goal in mind"]

[nan, nan, nan, ["that is preventing and they 're being prevented"], "that is preventing and they 're being prevented"]

[nan, nan, nan, ['from getting there .'], 'from getting there .']

['h6sm47j-Am444', 4.24, 4.27, ['So adding a little bit of a humorous goal'], 'So adding a little bit of a humorous goal']

[nan, nan, nan, ['to your story can make that story more lighthearted .'], 'to your story can make that story more lighthearted .']

['h6sm47j-Am445', 4.3, 4.3, ['So 

In [20]:
#### 
# Write the annotation result to Excel File

import xlrd
import xlsxwriter
workbook = xlsxwriter.Workbook('PresentationCorpusAnnotatedUsingBertBaseUncaseWithFullStop-BinaryStructure.xlsx')
worksheet = workbook.add_worksheet('Sheet1')
worksheet.write('A1', 'SegmentId')
worksheet.write('B1', 'StartTime')
worksheet.write('C1', 'EndTime')
worksheet.write('D1', 'Transcript')
worksheet.write('E1', 'Topic')
row=1
col=0
transRow=0

for i in allSegmentsTrans:
    print(i[0])
    print(i[4])
    print()
    print('row= ',row)
    print()
    mainRow=row
    worksheet.write(row,col,str(i[0]))
    worksheet.write(row,col+1,str(i[1]))
    worksheet.write(row,col+2,str(i[2]))
    transRow=row
    for j in i[3]:
        worksheet.write(row,col+3,str(j))
        row+=1
    print('prediction= ',predictor.predict(i[4]))
    print()
    worksheet.write(mainRow,col+4,predictor.predict(str(i[4])))
        
        

    
workbook.close()   


MnIPpUiTcRc1
Pay practitioner today, I'm going to be

row=  1

prediction=  No

nan
teaching you with the help of my friend,

row=  2

prediction=  No

nan
Dana Blouin, the top seven tip for a

row=  3

prediction=  No

nan
great presentation,

row=  4

prediction=  No

nan
Dana blue and a an entrepreneur

row=  5

prediction=  No

nan
currently living in Bangkok, Thailand,

row=  6

prediction=  No

nan
where he is a co and CEO of a

row=  7

prediction=  No

nan
company called mango lab. Anyways here's

row=  8

prediction=  No

MnIPpUiTcRc2
some of his tip. We all have to give

row=  9

prediction=  No

MnIPpUiTcRc3
presentation at different point in our

row=  10

prediction=  No

nan
life that might be for school or work or

row=  11

prediction=  Structure

nan
an organization that you weren't

row=  12

prediction=  Structure

nan
involved with no matter where We give

row=  13

prediction=  No

nan
the presentation, or to who we want to

row=  14

prediction=  No

nan
make sure

prediction=  Structure

nan
something Dana learned early on when he

row=  115

prediction=  No

nan
started his public speaking career.

row=  116

prediction=  No

MnIPpUiTcRc22
Another speaker at a big regional tech

row=  117

prediction=  No

nan
event came up to him after a talk and

row=  118

prediction=  Structure

nan
complimented Dana on is. delivery in the

row=  119

prediction=  No

MnIPpUiTcRc23
depth of the material but asked why he

row=  120

prediction=  No

nan
use more repetition Dana wa a

row=  121

prediction=  No

nan
bit stumped, because he never really

row=  122

prediction=  No

nan
thought about that before the Other

row=  123

prediction=  No

nan
speaker suggested that Dana start

row=  124

prediction=  No

nan
telling the Audience what it is. he

row=  125

prediction=  No

MnIPpUiTcRc24
wanted to tell them just to give them a

row=  126

prediction=  No

nan
brief rundown of the key point after

row=  127

prediction=  Structure

nan
that move right 

nan
and go through it you have to practice

row=  225

prediction=  Structure

nan
it in front of Other people Invite some

row=  226

prediction=  No

nan
friend over and have them give some

row=  227

prediction=  No

nan
feedback on your presentation and make

row=  228

prediction=  No

nan
improvement based on their feedback No

row=  229

prediction=  No

nan
one get it right the first time. But

row=  230

prediction=  Structure

MnIPpUiTcRc55
practice will

row=  231

prediction=  No

nan
get you towards perfect. With these seven

row=  232

prediction=  No

MnIPpUiTcRc56
tip you should be able to put together

row=  233

prediction=  Structure

nan
and deliver a masterful presentation

row=  234

prediction=  No

nan
that will wow your audience. and make

row=  235

prediction=  No

MnIPpUiTcRc57
sure that they remember you and your

row=  236

prediction=  Structure

nan
topic, If you guy, like this

row=  237

prediction=  No

nan
collaboration, be sure to check out

row=  

nan
because they feel confident and trusting

row=  327

prediction=  No

nan
and they believe that your project will be successful

row=  328

prediction=  Structure

nan
and that it is a good investment .

row=  329

prediction=  No

yoD8RMq2OkU41
All of that is not information it ’ s transformation .

row=  330

prediction=  Structure

yoD8RMq2OkU42
So it ’ s easy for me to say “ Don ’ t inform transform ! ”

row=  331

prediction=  No

nan
but how ?

row=  332

prediction=  Structure

nan
Well every magician need a magic wand and here is yours .

row=  333

prediction=  No

yoD8RMq2OkU43
It ’ s called the  Audience Transformation Roadmap . ''

row=  334

prediction=  Structure

yoD8RMq2OkU44
This is a really simple but amazingly powerful technique

row=  335

prediction=  No

nan
to understand what your transformational objective are

row=  336

prediction=  No

nan
and then to brainstorm

row=  337

prediction=  Structure

nan
what you can say show and do to transform your audienc

nan
about the speaker and the transformation .

row=  419

prediction=  No

yoD8RMq2OkU79
They need to know presentation are bad at information

row=  420

prediction=  No

nan
so maybe I should tell them that we forget most of what we hear very quickly .

row=  421

prediction=  No

yoD8RMq2OkU80
And perhaps I can find a visual way to show that a well .

row=  422

prediction=  No

yoD8RMq2OkU81
What next ? They need to know how to use the Roadmap so I ’ d better show it to them

row=  423

prediction=  No

nan
and I guess I could make it even clearer by going through an example .

row=  424

prediction=  Structure

yoD8RMq2OkU82
They need to believe

row=  425

prediction=  No

nan
that the Roadmap can help them to succeed in their presentation .

row=  426

prediction=  Structure

yoD8RMq2OkU83
So I ’ ll need to show how easy it is

row=  427

prediction=  No

nan
to take all of these sticky note off of the Transformation Column

row=  428

prediction=  No

nan
and arrange them in a

...

prediction=  Structure

h6sm47j-Am441
They 're trying to reach a destination

row=  6215

prediction=  Structure

nan
achieve something and that way your story get humorous

row=  6216

prediction=  Structure

nan
because the obstacle along the way prevent them

row=  6217

prediction=  No

nan
from reaching that goal and it becomes

row=  6218

prediction=  Structure

nan
pretty lighthearted pretty funny .

row=  6219

prediction=  No

h6sm47j-Am442
That 's the whole reason why a movie like Dumb and Dumber

row=  6220

prediction=  No

nan
is funny because it 's a road trip movie

row=  6221

prediction=  No

nan
where these two guy are trying to get from A to B

row=  6222

prediction=  Structure

nan
from Providence to Aspen to see about a girl

row=  6223

prediction=  No

nan
and everything that can go wrong doe go wrong

row=  6224

prediction=  Structure

nan
on their way .

row=  6225

prediction=  No

h6sm47j-Am443
And so all these mishap become very humorous

row=  6226

pred

prediction=  No

i68a6M5FFBc&t28
his presentation . And a if that were n't enough

row=  6303

prediction=  No

i68a6M5FFBc&t29
his pocketbook doe n't mind either . This ha been Powerful Presentations : Simply Stated .

row=  6304

prediction=  No

i68a6M5FFBc&t30
nan

row=  6305

prediction=  No

Yl_FJAOcFgQ1
To give a successful presentation, they

row=  6306

prediction=  Structure

nan
say you need to have a good beginning, a

row=  6307

prediction=  Structure

nan
good ending and keep them close together

row=  6308

prediction=  Structure

nan
and sure enough research show that

row=  6309

prediction=  No

nan
audience. Remember the first and last

row=  6310

prediction=  Structure

Yl_FJAOcFgQ2
few minute of a presentation, long after

row=  6311

prediction=  No

nan
forgotten most of what wa said

row=  6312

prediction=  No

nan
in the middle psychologist. Call this

row=  6313

prediction=  No

Yl_FJAOcFgQ3
the primacy recency effect, but you might

row=  6314

prediction

prediction=  Structure

nan
watch. These presenter clinch the

row=  6413

prediction=  No

Yl_FJAOcFgQ32
clothes, to summarise. Whenever we have

row=  6414

prediction=  Structure

Yl_FJAOcFgQ33
offered bonus to incentivize our staff

row=  6415

prediction=  Structure

nan
in sale, HR and manufacturing division,

row=  6416

prediction=  No

nan
productivity ha increased in some case

row=  6417

prediction=  No

nan
quite dramatically, but a we saw In R & D.

row=  6418

prediction=  No

Yl_FJAOcFgQ34
Introducing pay by performance ha had

row=  6419

prediction=  Structure

nan
precisely the opposite effect.

row=  6420

prediction=  No

Yl_FJAOcFgQ35
Incentivized research unit were, on

row=  6421

prediction=  No

nan
average only half a productive, a those

row=  6422

prediction=  No

nan
working without added incentive. What

row=  6423

prediction=  No

Yl_FJAOcFgQ36
are we to make of this Well, quite simply,

row=  6424

prediction=  Structure

nan
it seems bonus really do 

prediction=  No

RHX-xnP_G5s26
you offer number and statistic make

row=  6524

prediction=  Structure

nan
them meaningful, we have sold four

row=  6525

prediction=  Structure

nan
million iPhones to date. divided

row=  6526

prediction=  No

RHX-xnP_G5s27
four million by two hundred day.

row=  6527

prediction=  No

RHX-xnP_G5s28
20000 iPhones every day on average

row=  6528

prediction=  No

nan
number, mean much unless

row=  6529

prediction=  No

nan
placed in context manager connect the

row=  6530

prediction=  No

nan
dot for your listener Recently I

row=  6531

prediction=  No

nan
worked with a company, that launched a 12

row=  6532

prediction=  No

nan
gigabyte memory card 12 gigabyte that

row=  6533

prediction=  No

nan
number mean much to most people

row=  6534

prediction=  No

nan
So we put into context we said

row=  6535

prediction=  Structure

nan
enough memory to listen. to your music,

row=  6536

prediction=  No

RHX-xnP_G5s29
while traveling to the mo

prediction=  No

nan
you might tap on it or make other

row=  6636

prediction=  No

nan
distracting noise. So if you do need to

row=  6637

prediction=  No

7MWaeOHDBOg9
use the lectern for your note, make sure

row=  6638

prediction=  No

nan
that you are not touching it and that

row=  6639

prediction=  No

nan
you are gesturing. talked about

row=  6640

prediction=  No

7MWaeOHDBOg10
your stance. now, talk about your

row=  6641

prediction=  No

7MWaeOHDBOg11
sound when you are giving a speech. that

row=  6642

prediction=  No

7MWaeOHDBOg12
great stance is the basis for phenomenal

row=  6643

prediction=  No

nan
projection,

row=  6644

prediction=  No

nan
it will allow you to speak from your

row=  6645

prediction=  No

nan
diaphragm and fill the room with sound

row=  6646

prediction=  No

nan
make sure that you really lower those

row=  6647

prediction=  No

nan
shoulder, because when We get nervous

row=  6648

prediction=  No

nan
which most of U are when We speak